In [162]:
## PART 1
import re
import numpy as np
import datetime

# read file
file = open("input.txt","r")

# check file mode and readlines
if file.mode == 'r':
    data = file.read().splitlines()
    
# sort data in right order
data.sort()

# set variables
c_data = []
res_array = []
minutes = np.zeros(60)

# loop through data and create data line per shift 
for i in range(int(len(data))):
    if re.match(r'(.*)Guard #[0-9]+ begins shift', data[i]) is not None:
        c_data.append([guard_id,minutes.copy(),tot_min_asleep])
        guard = re.match(r'\[[0-9]{4}-(?P<month>[0-9]{2})-(?P<day>[0-9]{2}) (?P<hour>[0-9]{2}):(?P<minute>[0-9]{2})\] Guard #(?P<id>[0-9]+) begins shift', data[i]).groupdict()
        guard_id = int(guard["id"])
        minutes[0:60] = 0
        tot_min_asleep = 0
    elif re.match(r'(.*)falls asleep', data[i]) is not None:
        asleep = re.match(r'\[[0-9]{4}-(?P<month>[0-9]{2})-(?P<day>[0-9]{2}) (?P<hour>[0-9]{2}):(?P<minute>[0-9]{2})\] falls asleep', data[i]).groupdict()   
    elif re.match(r'(.*)wakes up', data[i]) is not None:
        wakes = re.match(r'\[[0-9]{4}-(?P<month>[0-9]{2})-(?P<day>[0-9]{2}) (?P<hour>[0-9]{2}):(?P<minute>[0-9]{2})\] wakes up', data[i]).groupdict()
        minutes[int(asleep["minute"]):int(wakes["minute"])] = 1
        tot_min_asleep += int(wakes["minute"]) - int(asleep["minute"])

# enter in dataframe and calculate sleeping time per guard
df = pd.DataFrame(c_data, columns=["id","minutes","min_asleep"])
df_group = df.groupby(['id']).sum()

# get guard with longest sleeping time
max_id = df_group.loc[df_group['min_asleep'].idxmax()].name

# calculate minute with most sleep
df_guard = df.loc[df['id']==max_id]
minutes_count = np.zeros(60)
for index, row in df_guard.iterrows():
    min_temp = row["minutes"]
    for i in range(len(row["minutes"])):
        if row["minutes"][i] > 0:
            minutes_count[i] += 1
minute_with_most_sleep = np.where(minutes_count == np.amax(minutes_count))[0][0]

# Calculate answer
print(minute_with_most_sleep*max_id)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
48680


In [231]:
## PART 2
mins = df['minutes'].apply(pd.Series)
df_2 =pd.concat([df[:], mins[:]], axis=1)
df_2 = df_2.drop(["minutes","min_asleep"], axis=1)
df_2 = df_2.set_index("id")
df_2 = df_2.groupby(['id']).sum()
max_minute = np.where(df_2.values.max(0) == df_2.values.max())[0][0]
guard_id = df_2.index[np.where(df_2.values.max(1) == df_2.values.max())[0][0]]
max_minute*guard_id

94826